In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.stats import multivariate_normal, kstest

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

import pymc as pm
import arviz as az

#import aesara.tensor as at

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

#import pyswarms as ps

######################################
#from pmdarima import auto_arima
#from diptest import diptest



In [ ]:
class PortfolioData:
  def __init__(self, rfr, dist, index_name):
    self.rfr = rfr
    self.dist = dist
    self.index_name = index_name
    self.pf = None
    self.index = None
    self.mu_index = None
    self.sigma_index = None
    self.mu_vector = None
    self.cov_matrix = None
    self.dof_t = None
    self.log_returns = None
    self.log_returns_index = None

  def get_data(self):
    filename_1 = 'nifty50_database.db'
    ffilename_1 = '/content/drive/My Drive/LEARN/Finance/quantitative finance/Portfolio-optimization/nifty50_indices.xlsx'
    pf_1 = (DownloadData(filename_1, ffilename_1).query_all_stocks())
    index_1 = (DownloadData(ffilename_1, ffilename_1).read_indices())

    filename_2 = 'niftynext50_database.db'
    ffilename_2 = '/content/drive/My Drive/LEARN/Finance/quantitative finance/Portfolio-optimization/niftynext50_indices.xlsx'
    pf_2 = (DownloadData(filename_2, ffilename_2).query_all_stocks())
    index_2 = (DownloadData(ffilename_2, ffilename_2).read_indices())

    if self.index_name == 'nifty50':
        self.pf = pf_1.copy()
        self.index = index_1.copy()
    if self.index_name == 'niftynext50':
        self.pf = pf_2.copy()
        self.index = index_2.copy()

    self.logreturn_index(index_1, index_2)

    return (pd.DataFrame({'Index': range(len(self.pf.columns)), 'Column Name': self.pf.columns}))

  def logreturn_index(self, index1, index2):
      index = index1[['Close']].copy()
      index.rename(columns={'Close': 'nifty50'}, inplace=True)
      index ['niftynext50'] = index2['Close']

      pfa = pf_analysis(index, self.dist)
      pfa.check_plot()

      self.log_returns_index = pfa.log_return_val()
      mu_vector, cov_matrix, dof_t = pfa.fit_joint(self.log_returns_index)

      self.mu_index, self.sigma_index = mu_vector[0], cov_matrix[0,0]

  def logreturn_portfolio(self, col_name):
      pf = self.pf.iloc[:, col_name].copy()
      self.pf = pf.copy()

      pfa = pf_analysis(self.pf, 't')
      pfa.check_plot()
      pfa.CAGR()

      self.log_returns = pfa.log_return_val()

      if self.dist == 't':
          #self.mu_vector, self.cov_matrix, self.dof_t = pfa.fit_marginal(self.log_returns)
          self.mu_vector, self.cov_matrix, self.dof_t = pfa.fit_joint(self.log_returns)
      elif self.dist == 'norm':
          #self.mu_vector, self.cov_matrix = pfa.fit_marginal(self.log_returns)
          self.mu_vector, self.cov_matrix = pfa.fit_joint(self.log_returns)

  def MPT(self, opt_optimize):
    ef = EfficientFrontier(self.rfr, self.mu_vector, self.cov_matrix, list(self.pf.columns))
    ef.plot_oppurtunity_set()  #n risky assets only
    ef.compute_frontier(opt_optimize) # minvar, maxret, cml

  def Factor(self, opt_model, opt_method):
    if opt_model == 'CAPM':
        FM =FactorModels(self.rfr, self.log_returns, self.log_returns_index[[self.index_name]].copy())

        if opt_method == 'OLS':
          FM.CAPM_OLS(self.dist)
        elif opt_method == 'GLS':
          FM.CAPM_GLS(self.dist)
        #elif opt_method == 'MLR'
          #FM.CAPM_MLR(self.dist)
